# Goals:
- Usable taz files
- usable skims
- trip tables

In [33]:
import pandas as pd
import numpy as np
import openmatrix as omx
import glob
import re

In [34]:
def store_omx_as_dict(infile_path):
    store_dict = {}
    with omx.open_file(infile_path,'r') as f:
        for name in f.list_matrices():
            store_dict[name] = np.array(f[name])
    return store_dict

In [37]:
# trip tables
# drop zone with SPRT_ID = 300, trip table index is 210

purpose_dict = {
    'HBW': 'HBW',
    'HBPB':'HBO',
    'HBSR':'HBO',
    'NHBO':'NHB',
    'NHBW':'NHB',
    'HBSc_preKto8':'HBSc1',
    'HBSc_HighSchool':'HBSc2',
    'HBSc_CollegeComm':'HBSc3'
}

modes = ['Bike','Walk', 'WAT', 'SOV', 'HOV2p', 'HOV3p',
        'DAT_Boat','DAT_CR','DAT_LB','DAT_RT',
        'DET_Boat','DET_CR','DET_LB','DET_RT']

# pre-mode-choice trip tables: eliminate unused zone, pre-mode choice, combine purposes (HBW, HBO, NHB, HBSc1, HBSc2, HBSc3), fix matrix names
for year in ['2018','2040']:
    out_dir = r'C:\Users\xchang\Documents\Seaport_data\data\\' + year
    if year == '2018':
        in_dir = 'X:/Share/OAK/Transfer/XChang/2018 Base/OMX/'
    else: in_dir = 'X:/Share/OAK/Transfer/XChang/2040 No Build/OMX/'
    
    skim = {}
    
    raw = {}
    
    # market segments: PK = AM+PM, OP = MD+NT; 1: withAutoOnly, 0: wAuto - withAutoOnly
    # format: 0_PK
    for tod in ['AM','MD','PM','NT']:
        tt1_tables = {}
        tt0_tables = {}
        tt1 = store_omx_as_dict(glob.glob(in_dir+ f'Seaport_Final*{tod}*withAutoOnly*.omx')[0])
        tt_all = store_omx_as_dict(glob.glob(in_dir+ f'Seaport_Final*{tod}*Zero*.omx')[0])
        for purpose in purpose_dict:
            try:
                tt1_tables[purpose_dict[purpose]] += sum([tt1[f"Sum of '{purpose}_{mode}'"] for mode in modes])
            except: tt1_tables[purpose_dict[purpose]] = sum([tt1[f"Sum of '{purpose}_{mode}'"]  for mode in modes])
            
            try:
                tt0_tables[purpose_dict[purpose]] += sum([tt_all[f"Sum of '{purpose}_{mode}'"]  for mode in modes]) - sum([tt1[f"Sum of '{purpose}_{mode}'"] for mode in modes])
            except: tt0_tables[purpose_dict[purpose]] = sum([tt_all[f"Sum of '{purpose}_{mode}'"]  for mode in modes]) - sum([tt1[f"Sum of '{purpose}_{mode}'"] for mode in modes])
        raw[tod] = {'0Auto': tt0_tables, 'wAuto': tt1_tables}
    

    raw['PK'] = {'0Auto':{purpose: (raw['AM']['0Auto'][purpose] + raw['PM']['0Auto'][purpose]) for purpose in tt0_tables}, 'wAuto':{purpose: (raw['AM']['wAuto'][purpose] + raw['PM']['wAuto'][purpose]) for purpose in tt0_tables}}
    raw['OP'] = {'0Auto':{purpose: (raw['MD']['0Auto'][purpose] + raw['NT']['0Auto'][purpose]) for purpose in tt0_tables}, 'wAuto':{purpose: (raw['MD']['wAuto'][purpose] + raw['NT']['wAuto'][purpose]) for purpose in tt0_tables}}
    
    # format: purpose_peak_vehicle
    for peak in ['PK','OP']:
        for veh in raw[peak]:
            for purpose in raw[peak][veh]:
                skim[f'{purpose}_{peak}_{veh}'] = raw[peak][veh][purpose]
                
    with omx.open_file(out_dir+'/pre_MC_trip_6_purposes.omx', 'w') as tt:
        for key in skim:
            tt[key] = np.delete(np.delete(skim[key],210,0), 210, 1)

In [121]:
# skims
skim_terms =  {'AM_A_DAT_for_Boat_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'AM_A_DAT_for_CommRail_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'AM_A_DAT_for_LocalBus_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'AM_A_DAT_for_Rapid_Transit_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'AM_SOV_skim_SPRTDST.omx' : ['Auto_Toll (Skim)', 'CongTime', 'Length (Skim)', 'TerminalTimes'],
 'AM_WAT_for_All_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'MD_A_DAT_for_Boat_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'MD_A_DAT_for_CommRail_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'MD_A_DAT_for_LocalBus_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'MD_A_DAT_for_Rapid_Transit_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'MD_SOV_skim_SPRTDST.omx' : ['Auto_Toll (Skim)', 'CongTime', 'Length (Skim)', 'TerminalTimes'],
 'MD_WAT_for_All_tr_skim_SPRTDST.omx' : ['Total_Cost', 'Total_IVTT', 'Total_OVTT'],
 'WalkSkim_SPRTDST.omx' : ['Length (Skim)', 'OneMileorLess', 'WalkTime'],
 'BikeSkim_SPRTDST.omx' : ['BikeTime', 'Length (Skim)', 'OneMileorLess']}

in_dir = 'X:/Share/OAK/Transfer/XChang/2018 Base/OMX/'
out_dir = r'C:\Users\xchang\Documents\Seaport_data\2018\skims\\'
for fn in skim_terms:
    with omx.open_file(in_dir + fn, 'r') as in_skim, omx.open_file(out_dir + fn, 'w') as out_skim:
        for term in skim_terms[fn]:
            out_skim[term] = in_skim[f"Sum of '{term}'"]
            
in_dir = 'X:/Share/OAK/Transfer/XChang/2040 No Build/OMX/'
out_dir = r'C:\Users\xchang\Documents\Seaport_data\2040\skims\\'
for fn in skim_terms:
    with omx.open_file(in_dir + fn, 'r') as in_skim, omx.open_file(out_dir + fn, 'w') as out_skim:
        for term in skim_terms[fn]:
            out_skim[term] = in_skim[f"Sum of '{term}'"]

C:\Users\xchang\AppData\Local\Continuum\anaconda3\lib\site-packages\tables\path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'Auto_Toll (Skim)'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\xchang\AppData\Local\Continuum\anaconda3\lib\site-packages\tables\path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'Length (Skim)'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [62]:
# TAZ 2018
# drop zone with SPRT_ID = 300, trip table index is 210

skim_mapping = pd.read_csv(r"C:\Users\xchang\OneDrive - Cambridge Systematics\Documents\Seaport\skim_zone_mapping.csv")

TAZ_crosswalk = pd.read_csv(r"X:\Share\OAK\Transfer\XChang\MA_TAZs_with_Seaport_table.txt")[['MA_TAZ_ID','SPRT_ID']]
in_dir = r'C:\Users\xchang\Documents\FM_data\data\2016\\'
out_dir = r'C:\Users\xchang\Documents\Seaport_data\2018\\'
#zonal
f = 'taz_zonal.csv'
df = pd.read_csv(in_dir + f).merge(TAZ_crosswalk, left_on = 'ID', right_on = 'MA_TAZ_ID').groupby('SPRT_ID').agg(
    {'Area': 'sum', 'Tot_Pop': 'sum', 'Tot_Emp':'sum', 'HH_Pop':'sum','HH':'sum','VehiclesPerWorker':'mean',
    'AM_wacc_fact':'mean','MD_wacc_fact':'mean','Hwy Prod Term Time':'mean'})
df.drop(300, inplace = True)
df.to_csv(out_dir + f)


In [78]:
# parking
f = "Land_Use_Parking_Costs.csv"
parking = pd.read_csv(in_dir + f).merge(TAZ_crosswalk, left_on = 'ID', right_on = 'MA_TAZ_ID').groupby('SPRT_ID').mean()['Daily Parking Cost']

df = pd.DataFrame(0, index = skim_mapping['SPRT_ID'], columns = ['Daily Parking Cost'])
df['Daily Parking Cost'] = parking
df.fillna(0).to_csv(out_dir + f)

In [100]:
# taz

f = 'SW_TAZ_2010.csv'
df = pd.read_csv(in_dir + f).merge(TAZ_crosswalk, left_on = 'ID_FOR_CS', right_on = 'MA_TAZ_ID').groupby('SPRT_ID').agg(
{'BOSTON_NB':'first', 'TOWN': lambda x: '; '.join(x.unique())})
df.to_csv(out_dir + f)

In [117]:
# taz interstate
interstate = pd.read_csv(r"C:\Egnyte\Shared\Projects\190077\1 - Existing Conditions\4 - CTPS\gis\jayne\TAZ_SPRT_interstate.csv")

In [120]:
interstate.drop('FID',axis = 1).set_index('SPRT_ID').to_csv(r'C:\Users\xchang\Documents\Seaport_data\taz_interstate.csv')

In [122]:
# TAZ 2040
# drop zone with SPRT_ID = 300, trip table index is 210

skim_mapping = pd.read_csv(r"C:\Users\xchang\OneDrive - Cambridge Systematics\Documents\Seaport\skim_zone_mapping.csv")

TAZ_crosswalk = pd.read_csv(r"X:\Share\OAK\Transfer\XChang\MA_TAZs_with_Seaport_table.txt")[['MA_TAZ_ID','SPRT_ID']]
in_dir = r'C:\Users\xchang\Documents\FM_data\data\2040\\'
out_dir = r'C:\Users\xchang\Documents\Seaport_data\2040\\'

#zonal
f = 'TAZ_zonal_2040.csv'
df = pd.read_csv(in_dir + f).merge(TAZ_crosswalk, left_on = 'ID', right_on = 'MA_TAZ_ID').groupby('SPRT_ID').agg(
    {'Area': 'sum', 'Acc_PEV':'mean','Egr_PEV':'mean','Tot_Pop': 'sum', 'Tot_Emp':'sum', 'HH_Pop':'sum','HH':'sum','VehiclesPerWorker':'mean',
    'AM_wacc_fact':'mean','MD_wacc_fact':'mean','Hwy Prod Term Time':'mean'})
df.drop(300, inplace = True)
df.to_csv(out_dir + f)


# parking
f = "Land_Use_Parking_Costs.csv"
parking = pd.read_csv(in_dir + f).merge(TAZ_crosswalk, left_on = 'ID', right_on = 'MA_TAZ_ID').groupby('SPRT_ID').mean()['Daily Parking Cost']
df = pd.DataFrame(0, index = skim_mapping['SPRT_ID'], columns = ['Daily Parking Cost'])
df['Daily Parking Cost'] = parking
df.fillna(0).to_csv(out_dir + f)




In [66]:
TAZ_crosswalk = pd.read_csv(r"X:\Share\OAK\Transfer\XChang\MA_TAZs_with_Seaport_table.txt")


# taz

f = 'SW_TAZ_2010.csv'

# want TAZ_crosswalk where SPRT_ID is in skim_mapping['SPRT_ID']
# keep town information in TAZ_crosswalk
# merge BOSTON_NB info from f

town = skim_mapping[['SPRT_ID']].merge(TAZ_crosswalk.groupby('SPRT_ID').first()[['TOWN']].reset_index())
nb = pd.read_csv(r"X:\Share\OAK\Transfer\XChang\MA_TAZs_with_Seaport_table.txt")[['MA_TAZ_ID','SPRT_ID','TOWN']].merge(
    pd.read_csv(in_dir + f)[['ID_FOR_CS','BOSTON_NB']], left_on = 'MA_TAZ_ID', right_on = 'ID_FOR_CS').groupby('SPRT_ID').first()['BOSTON_NB'].reset_index()

df = town.merge(nb, how = 'left')
df.to_csv(out_dir + f)